# Financial Data Preprocessing
A time series is a series of data points indexed in time order. Financial Data such as equity, commodity, and forex price series observed at equally spaced points in time are an example of such a series. It is a sequence of data points observed at regular time intervals and depending on the frequency of observations, a time series may typically be in ticks, seconds, minutes, hourly, daily, weekly, monthly, quarterly and annual.

The first step towards any data analysis would be to parse the raw data that involves extracting the data from the source and then cleaning and filling the missing data if any. While data comes in many forms, Python makes it easy to read time-series data using useful packages.

In this session, we will retrieve and store both end-of-day and intraday data using some of the popular python packages. These libraries aim to keep the API simple and make it easier to access historical data. Further, we will see how to read data from traditional data sources stored locally.

In [ ]:
# Ignore warnings - optional
import warnings
warnings.filterwarnings('ignore')

# Import data manipulation libraries
import numpy as np
import pandas as pd
        
# Import yahoo finance library
import yfinance as yf

# Data Retrieval
Retrieving EOD, Intraday, Options data

Retrieving end-of-day data for single security
We'll retrieve historical data from yahoo finance using yfinance library

In [ ]:
# Example 1
# Fetch the data by specifying the number of period
df1 = yf.download("AAPL", period="10d", progress=False)

# Display the first five rows of the dataframe to check the results. 
df1.head()